In [1]:
# !pip install sqlite3
# !pip install time
# !pip install psutil
# !pip install os
# !pip install faker

In [2]:
import sqlite3
import time
import random
import psutil
import os
from faker import Faker
fake = Faker("pt_br")

In [3]:
# Função para monitorar memória e CPU
def monitorar_recursos():
    """
    Retorna o uso de memória e CPU do sistema no momento da execução.

    Returns:
        memoria (float): Uso de memória em MB.
        cpu (float): Uso da CPU em %.
    """
    processo = psutil.Process(os.getpid())
    memoria = processo.memory_info().rss / (1024 * 1024)  # Memória em MB
    cpu = psutil.cpu_percent(interval=1)  # Uso de CPU em %
    return memoria, cpu


# Função para medir tempo de execução
def medir_tempo(func):
    """
    Decorador para medir o tempo de execução de uma função e monitorar o consumo de memória e CPU.

    Args:
        func (function): Função a ser medida.

    Returns:
        function: Retorna a função decorada com monitoramento de tempo e recursos.
    """

    def wrapper(*args, **kwargs):
        inicio = time.time()
        method, commit_type = func(*args, **kwargs)
        fim = time.time()
        tempo_total = fim - inicio
        memoria, cpu = monitorar_recursos()
        return method, commit_type, tempo_total, memoria, cpu

    return wrapper


def registrar_resultados(
    tempo_processamento: float,
    memoria: float,
    cpu: float,
    nr_registros: int,
    method: str,
    commit_type: str,
    conexao: sqlite3.Connection,
):
    """
    Registra os resultados de desempenho na tabela 'results'.

    Args:
        tempo_processamento (float): Tempo de execução da operação.
        memoria (float): Uso de memória em MB.
        cpu (float): Uso de CPU em %.
        nr_registros (int): Número de registros inseridos.
        method (str): Método de inserção ('execute' ou 'executemany').
        commit_type (str): Tipo de commit ('batch' ou 'per_row').
        conexao (sqlite3.Connection): Conexão ativa com o banco de dados.
    """
    cursor_registro = conexao.cursor()
    cursor_registro.execute(
        "insert into t_sqlite_insert_results (time, memori, cpu, nr_registros, method, commit_type) values (?,?,?,?,?,?)",
        (tempo_processamento, memoria, cpu, nr_registros, method, commit_type),
    )
    conexao.commit()


# Contar registros
def contar_registros(conexao):
    cursor = conexao.cursor()
    cursor.execute("SELECT COUNT(*) FROM t_sqlite_insert;")
    resultado = cursor.fetchone()
    print(f"Total de registros: {resultado[0]}")


# genarator numérico
def generator_range(nr_range: int):
    """
    Retorna um generator com o range começando em x + 1
    Desta forma um range(10) começa em 1 e termina em 10
    """
    return (x + 1 for x in range(nr_range))

In [4]:
nome_banco = "sqlite_database.bd"
test_range = 10
print(f"Banco de dados '{nome_banco}' criado.")
try:
    conexao = sqlite3.connect(nome_banco)
    cursor = conexao.cursor()
    cursor.executescript(
        """
        CREATE TABLE IF NOT EXISTS t_sqlite_insert (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT,
            valor INTEGER
        );
        CREATE TABLE IF NOT EXISTS t_sqlite_insert_many (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT,
            valor INTEGER
        );
        CREATE TABLE IF NOT EXISTS t_sqlite_insert_results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            time real,
            memori real,
            cpu real,
            nr_registros integer,
            method text,
            commit_type text CHECK(commit_type IN ('batch', 'per_row'))
        );
    """
    )
    conexao.commit()
    print("Tabelas criadas com sucesso.")
except Exception as e:
    print('Erro: ', e)
finally:
    conexao.close()

Banco de dados 'sqlite_database.bd' criado.
Tabelas criadas com sucesso.


Banco de dados usado em SQLite, separado em 3 tabelas diferentes.
- 1 para armazenar os resultados.
- 2 para os diferentes tipos de insert (execute e execute_many)

---

In [12]:
@medir_tempo
def inserir_dados_execute(conexao, num_registros):
    cursor = conexao.cursor()
    cursor.execute("BEGIN TRANSACTION;")
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        cursor.execute("INSERT INTO t_sqlite_insert (nome, valor) VALUES (?, ?)", (nome, valor))
    conexao.commit()
    return 'execute', 'per_row'

@medir_tempo
def inserir_dados_execute_many(conexao, num_registros):
    cursor = conexao.cursor()
    cursor.execute("BEGIN TRANSACTION;")
    lst_insert_many = list()
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        lst_insert_many.append((nome, valor))
        
    cursor.executemany("INSERT INTO t_sqlite_insert_many (nome, valor) VALUES (?, ?)", lst_insert_many)
    conexao.commit()
    return 'execute', 'batch'

Funções para inserir os dados de formas diferentes

---

In [6]:
# Testando o script
try:
    conexao = sqlite3.connect(nome_banco)
    for nr_insert in generator_range(test_range):
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute(
            conexao, nr_insert
        )

        registrar_resultados(
            tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao
        )
except Exception as e:
    print(e)
finally:
    conexao.close()
    print("Connection close")

Connection close


In [13]:
# Testando o script
try:
    conexao = sqlite3.connect(nome_banco)
    for nr_insert in generator_range(test_range):
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute_many(
            conexao, nr_insert
        )

        registrar_resultados(
            tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao
        )
except Exception as e:
    print(e)
finally:
    conexao.close()
    print("Connection close")

Connection close


In [8]:
try:
    conexao = sqlite3.connect(nome_banco)

    for nr_insert in generator_range(test_range):
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute(conexao, nr_insert)
        registrar_resultados(tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao)

    for nr_insert in generator_range(test_range):
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute_many(conexao, nr_insert)
        registrar_resultados(tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao)

except Exception as e:
    print("ERRO:", e)
finally:
    conexao.close()
    print("Conexão fechada com sucesso.")


ERRO: no such table: t_sqlite_insert_batch
Conexão fechada com sucesso.


loop for para inserir os dados seguindo uma progressão geométrica.

---

In [9]:
monitorar_recursos()

(75.3984375, 29.3)

In [10]:
# %%
with sqlite3.connect(nome_banco) as conexao:
    cursor = conexao.cursor()
    cursor.execute(
        "SELECT COUNT(nome) AS contagem_nome, SUM(valor) AS soma_valor FROM t_sqlite_insert WHERE nome LIKE 'F%'"
    )
    resultado = cursor.fetchall()

print("Resultados da consulta:", resultado)

Resultados da consulta: [(590008, 294656402851)]
